<a href="https://colab.research.google.com/github/blackkk466/chatbot_with_lstm/blob/master/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tflearn

In [0]:
from __future__ import absolute_import, division, print_function

import os
import pickle

import six
from six.moves import urllib

import tflearn
from tflearn.data_utils import *

In [0]:
path = "S1B1.txt"
char_idx_file = 'char_idx.pickle'
maxlen = 100
char_idx = None

X, Y, char_idx = textfile_to_semi_redundant_sequences(path, seq_maxlen=maxlen, redun_step=3, pre_defined_char_idx=char_idx)

pickle.dump(char_idx, open(char_idx_file, 'wb'))

In [0]:
q = tflearn.input_data([None, maxlen, len(char_idx)])
q = tflearn.lstm(q, 128, return_seq=True)
q = tflearn.dropout(q, 0.5)
q = tflearn.lstm(q, 128, return_seq=True)
q = tflearn.dropout(q, 0.5)
q = tflearn.lstm(q, 128)
q = tflearn.dropout(q, 0.5)

q = tflearn.fully_connected(q, len(char_idx), activation='softmax')
q = tflearn.regression(q, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.001)

m = tflearn.SequenceGenerator(q, dictionary=char_idx, seq_maxlen=maxlen, clip_gradients=5.0, checkpoint_path='model_mentalist')

In [0]:
for i in range(50):
  seed = random_sequence_from_textfile(path, maxlen)
  m.fit(X, Y, validation_set=0.1, batch_size=128, n_epoch=1, run_id='mentalist'),
  print("--TESTING...")
  print("--Test with temperature of 0.5 ---")
  print(m.generate(600, temperature=0.5, seq_seed=seed))